In [1]:
import matplotlib.pyplot as plt
import numpy as np
import cPickle

import analysis
reload(analysis)

Using gpu device 0: Tesla K40c (CNMeM is enabled with initial size: 20.0% of memory, cuDNN 4007)


<module 'analysis' from 'analysis.pyc'>

In [2]:
fname = '/home/jesse/Downloads/comparison_128_2.pkl'

W_priors = ['L2', 'L4', 'RANDOM']
ica_models = [2, 4, 6, 8, 'RANDOM', 'COULOMB', 'COHERENCE'] + ['SC']
lambdas = np.logspace(-2, 2, num=9)
colors = ['black', 'black', 'black', 'black', 'red', 'green', 'blue', 'purple']
fmts = ['-', '--', ':', '-.', '-', '-', '-', '-']

In [3]:
with open(fname, 'r') as f:
    data = cPickle.load(f)

In [4]:
A_dict, W_dict, W_fits, min_ks, results = data
n_mixtures, n_sources = A_dict[A_dict.keys()[0]][0].shape
n_iter = W_fits.shape[3]
results = np.nan * np.ones((len(W_priors), len(ica_models), lambdas.size, n_iter, 2))

In [5]:
for ii, p in enumerate(W_priors):
    for jj, m in enumerate(ica_models):
        A = A_dict[p][jj]
        for kk, l in enumerate(lambdas):
            for ll in range(n_iter):
                W = W_fits[ii, jj, kk, ll]
                results[ii, jj, kk, ll] = analysis.recovery_statistics_AW(A, W)

In [13]:
for ii, p in enumerate(W_priors):
    f, (ad, ae) = plt.subplots(1, 2, figsize=(16, 8))
    for jj, m in enumerate(ica_models):
        dist = results[ii, jj, :, :, 0].mean(axis=1)
        delta = results[ii, jj, :, :, 1].mean(axis=1)
        ad.semilogx(lambdas, dist, fmts[jj], label=str(m), c=colors[jj])
        ad.semilogx(lambdas, dist, 'o', c=colors[jj])
        ae.semilogx(lambdas, delta, fmts[jj], label=str(m), c=colors[jj])
        ae.semilogx(lambdas, delta, 'o', c=colors[jj])
    ad.set_title(p)
    ad.set_xlabel('Lambda')
    ad.set_ylabel('Hellinger Distance')
    ae.set_title(p)
    ae.set_xlabel('Lambda')
    ae.set_ylabel('Permutation Delta')
    ae.legend(loc='best')
    plt.savefig(p+'.pdf')
plt.show()
plt.close()